Wilcoxon Signed-Rank test

In [5]:
import math
from typing import Iterable, List, Tuple, Union

def _average_ranks_and_ties(abs_vals: List[float], tol: float = 0.0) -> Tuple[List[float], List[int]]:
    """
    Rank |x_i| from smallest to largest starting at 1, averaging ranks within ties.
    A tolerance 'tol' groups values whose differences are <= tol into the same tie block.
    Returns:
        ranks: list of average ranks in the original order
        tie_sizes: sizes (>1) of tie groups among |x_i|
    """
    n = len(abs_vals)
    pairs = sorted([(abs_vals[i], i) for i in range(n)], key=lambda z: z[0])

    ranks = [0.0] * n
    tie_sizes = []
    i = 0
    while i < n:
        j = i + 1
        v = pairs[i][0]
        while j < n and abs(pairs[j][0] - v) <= tol:
            j += 1
        m = j - i  # size of tie block
        # average of 1-based ranks: i+1, i+2, ..., j
        avg_rank = (i + 1 + j) * 0.5
        for k in range(i, j):
            ranks[pairs[k][1]] = avg_rank
        if m > 1:
            tie_sizes.append(m)
        i = j
    return ranks, tie_sizes

def _normal_cdf(z: float) -> float:
    return 0.5 * (1.0 + math.erf(z / math.sqrt(2.0)))

def _exact_wilcoxon_pvalue(W_obs: int, n: int, alternative: str) -> float:
    """
    Exact p-value for W+ when there are NO ties (ranks are exactly 1..n).
    Dynamic programming over subset sums of {1,2,...,n}.
    """
    S = n * (n + 1) // 2
    dp = [0] * (S + 1)
    dp[0] = 1
    for r in range(1, n + 1):
        for s in range(S, r - 1, -1):
            dp[s] += dp[s - r]

    total = 2 ** n  # each sign pattern equally likely under H0
    if alternative == "greater":
        return sum(dp[W_obs:]) / total
    elif alternative == "less":
        return sum(dp[: W_obs + 1]) / total
    else:  # two-sided (symmetric about mu)
        p_ge = sum(dp[W_obs:]) / total
        p_le = sum(dp[: W_obs + 1]) / total
        return min(1.0, 2.0 * min(p_ge, p_le))

def wilcoxon_signed_rank(
    returns: Iterable[float],
    alternative: str = "two-sided",
    exact: Union[str, bool] = "auto",
    zero_method: str = "wilcox",
    continuity: bool = True,
    exact_n_max: int = 50,
    tie_tol: float = 0.0,
):
    """
    From-scratch Wilcoxon Signed-Rank test (no SciPy).

    - Drops zeros ('wilcox')
    - Exact p-value via DP when there are no ties and n <= exact_n_max
    - Otherwise: normal approximation with tie-corrected variance
      and optional continuity correction.

    Returns a dict with keys: 'N','W_plus','mu','sigma','z','p_value','method','tie_sizes'.
    """
    if alternative not in {"two-sided", "greater", "less"}:
        raise ValueError("alternative must be 'two-sided', 'greater', or 'less'.")
    if zero_method != "wilcox":
        raise NotImplementedError("Only zero_method='wilcox' (drop zeros) is supported.")

    # 1) Drop exact zeros
    x = [float(v) for v in returns if float(v) != 0.0]
    n = len(x)
    if n == 0:
        raise ValueError("All observations are zero after filtering; test undefined.")

    # 2) Handle n == 1 exactly
    if n == 1:
        W_plus = 1.0 if x[0] > 0 else 0.0
        mu = 0.5
        sigma = math.sqrt(0.25)
        # With one nonzero, one-sided exact p = 0.5; two-sided exact p = 1.0
        if alternative == "two-sided":
            p = 1.0
        else:
            p = 0.5
        return {
            "N": n, "W_plus": W_plus, "mu": mu, "sigma": sigma,
            "z": float("nan"), "p_value": p, "method": "exact", "tie_sizes": []
        }

    # 3) Rank |x| with average ranks and tie detection (with tolerance)
    abs_x = [abs(v) for v in x]
    ranks, tie_sizes = _average_ranks_and_ties(abs_x, tol=tie_tol)

    # 4) Compute W+ (sum of ranks for positive x)
    W_plus = sum(r for r, v in zip(ranks, x) if v > 0)

    # 5) Mean and (tie-corrected) variance under H0
    mu = n * (n + 1) / 4.0
    var = n * (n + 1) * (2 * n + 1) / 24.0  # no-tie variance
    if tie_sizes:
        var -= sum(t * (t + 1) * (2 * t + 1) for t in tie_sizes) / 48.0
    sigma = math.sqrt(var)

    # 6) Exact vs normal route
    no_ties = (len(tie_sizes) == 0)
    use_exact = (exact is True) or (exact == "auto" and no_ties and n <= exact_n_max)
    method = "exact" if use_exact else "normal-approx"

    if use_exact:
        # When no ties, ranks are exactly 1..n; W+ is integer-valued
        W_obs_int = int(round(W_plus))
        p = _exact_wilcoxon_pvalue(W_obs_int, n, alternative)
        z = float("nan")
    else:
        # Degenerate variance corner case
        if sigma == 0.0:
            if no_ties and n <= exact_n_max:
                p = _exact_wilcoxon_pvalue(int(round(W_plus)), n, alternative)
                return {"N": n, "W_plus": W_plus, "mu": mu, "sigma": sigma,
                        "z": float("nan"), "p_value": p, "method": "exact",
                        "tie_sizes": tie_sizes}
            return {"N": n, "W_plus": W_plus, "mu": mu, "sigma": sigma,
                    "z": float("nan"), "p_value": float("nan"),
                    "method": method, "tie_sizes": tie_sizes}

        # Normal approximation with optional continuity correction
        cc = 0.5 if continuity else 0.0
        if alternative == "greater":
            z = (W_plus - mu - cc) / sigma
            p = 1.0 - _normal_cdf(z)
        elif alternative == "less":
            z = (W_plus - mu + cc) / sigma
            p = _normal_cdf(z)
        else:  # two-sided
            z = (abs(W_plus - mu) - cc) / sigma
            p = 2.0 * (1.0 - _normal_cdf(z))
            p = min(1.0, max(0.0, p))

    return {
        "N": n,
        "W_plus": W_plus,
        "mu": mu,
        "sigma": sigma,
        "z": z,
        "p_value": p,
        "method": method,
        "tie_sizes": tie_sizes,
    }

if __name__ == "__main__":
    import numpy as np

    rng = np.random.default_rng(42)
    n = 250
    # Student-t with df=3 (fat tails), ~1% daily vol, slight positive drift
    returns = 0.0005 + 0.01 * rng.standard_t(df=3, size=n)

    # Add illustrative outliers (robustness to extremes) and zeros (which are dropped)
    returns[5]  += 0.12
    returns[101] -= 0.15
    returns[10] = 0.0
    returns[80] = 0.0

    res = wilcoxon_signed_rank(returns, alternative="greater", exact="auto",
                               continuity=True, exact_n_max=50, tie_tol=0.0)
    print(f"N = {res['N']}, W+ = {res['W_plus']:.3f}, method = {res['method']}")
    print(f"mu = {res['mu']:.3f}, sigma = {res['sigma']:.5f}, z = {res['z']}")
    print(f"p-value = {res['p_value']:.6f}  (H1: median > 0)")

N = 248, W+ = 16000.000, method = normal-approx
mu = 15438.000, sigma = 1130.83199, z = 0.49653706534986475
p-value = 0.309758  (H1: median > 0)


Chi test of a signle variance

In [6]:
import math
from typing import Iterable, Union, Dict

# Chi-square CDF via regularized incomplete gamma
def _gammainc_lower_reg(s: float, x: float, eps: float = 1e-12, itmax: int = 200) -> float:
    """
    Regularized lower incomplete gamma P(s, x) = gamma(s, x) / Gamma(s).
    Uses series expansion when x < s+1, else Lentz's continued fraction for Q(s, x).
    """
    if x <= 0.0:
        return 0.0
    if s <= 0.0:
        raise ValueError("s must be > 0")

    # Helper: exp(s*log(x) - x - lgamma(s))
    def _prefactor(ss: float, xx: float) -> float:
        return math.exp(ss * math.log(xx) - xx - math.lgamma(ss))

    if x < s + 1.0:
        # Series expansion for P(s, x)
        term = 1.0 / s
        summ = term
        k = 1
        while k < itmax:
            term *= x / (s + k)
            summ += term
            if abs(term) < abs(summ) * eps:
                break
            k += 1
        return _prefactor(s, x) * summ
    else:
        # Continued fraction for Q(s, x) = 1 - P(s, x)
        # Lentz's algorithm for the CF of the complemented gamma
        tiny = 1e-300
        f = 1.0
        C = 1.0 / tiny
        D = 0.0
        a_k = 0.0  # will be set within loop
        b_k = x - s + 1.0
        D = b_k if abs(b_k) > tiny else tiny
        C = b_k if abs(b_k) > tiny else tiny
        D = 1.0 / D
        f = C * D
        for k in range(1, itmax + 1):
            a_k = -k * (k - s)
            b_k = b_k + 2.0
            # First part
            D = a_k * D + b_k
            if abs(D) < tiny:
                D = tiny
            C = b_k + a_k / C
            if abs(C) < tiny:
                C = tiny
            D = 1.0 / D
            delta = C * D
            f *= delta
            if abs(delta - 1.0) < eps:
                break
        Q = math.exp(s * math.log(x) - x - math.lgamma(s)) * f
        P = 1.0 - Q
        # Guard against tiny numerical drift
        if P < 0.0:
            P = 0.0
        if P > 1.0:
            P = 1.0
        return P

def chi2_cdf(x: float, df: int) -> float:
    """
    CDF of Chi-square(df) using P(s=df/2, x/2).
    """
    if df <= 0 or int(df) != df:
        raise ValueError("df must be a positive integer.")
    if x <= 0.0:
        return 0.0
    return _gammainc_lower_reg(0.5 * df, 0.5 * x)

# Single-variance chi-square test (one/two-sided)
def chi2_variance_test(
    returns: Iterable[float],
    sigma0_sq: float,
    alternative: str = "two-sided",
    ddof: int = 1
) -> Dict[str, Union[int, float, str]]:
    """
    Test H0: sigma^2 == sigma0_sq using Chi-square with df = n - ddof.
    Assumes i.i.d. normal returns under H0.

    Parameters
    ----------
    returns : iterable of float
        Return series (NaNs are ignored).
    sigma0_sq : float
        Hypothesized variance (e.g., (0.15/sqrt(252))**2 for 15% annual vol).
    alternative : {'two-sided','greater','less'}
        'greater' tests sigma^2 > sigma0_sq; 'less' tests sigma^2 < sigma0_sq.
    ddof : int
        Degrees-of-freedom used by the variance estimator (default 1 for unbiased).

    Returns
    -------
    dict with keys: n, df, s2, chi2, p_value, alternative
    """
    import math

    x = [float(v) for v in returns if v is not None and not math.isnan(float(v))]
    n = len(x)
    if n - ddof <= 0:
        raise ValueError("Not enough observations given ddof.")

    # Sample variance with given ddof
    mean = sum(x) / n
    s2 = sum((v - mean) ** 2 for v in x) / (n - ddof)

    df = n - ddof
    chi2 = df * s2 / sigma0_sq

    # Tail probabilities via chi-square CDF
    F = chi2_cdf(chi2, df)
    if alternative == "greater":
        p = 1.0 - F  # right tail
    elif alternative == "less":
        p = F        # left tail
    elif alternative == "two-sided":
        # Two-sided p as doubling the smaller tail probability
        p = 2.0 * min(F, 1.0 - F)
        p = min(1.0, max(0.0, p))
    else:
        raise ValueError("alternative must be 'two-sided','greater', or 'less'.")

    return {"n": n, "df": df, "s2": s2, "chi2": chi2, "p_value": p, "alternative": alternative}

if __name__ == "__main__":
    import numpy as np

    # Synthetic heavy-tailed daily returns (Student-t df=3) with small drift
    rng = np.random.default_rng(42)
    n = 250
    r = 0.0005 + 0.01 * rng.standard_t(df=3, size=n)

    # Hypothesized annualized volatility cap 15% -> daily variance
    sigma0_sq = (0.15 / np.sqrt(252.0)) ** 2

    # Run two-sided chi-square test of sigma^2 == sigma0_sq
    out = chi2_variance_test(r, sigma0_sq, alternative="two-sided", ddof=1)
    print(f"n={out['n']}, df={out['df']}")
    print(f"sample variance s^2 = {out['s2']:.8f}")
    print(f"chi^2 stat = {out['chi2']:.4f}, p-value = {out['p_value']:.6f} ({out['alternative']})")

n=250, df=249
sample variance s^2 = 0.00035838
chi^2 stat = 999.4510, p-value = 0.000000 (two-sided)


Levene's test

In [7]:
import math
from typing import List, Sequence, Dict, Union

# Regularized incomplete beta via continued fraction
def _betacf(a: float, b: float, x: float, itmax: int = 200, eps: float = 1e-12) -> float:
    am, bm = 1.0, 1.0
    az = 1.0
    qab = a + b
    qap = a + 1.0
    qam = a - 1.0
    bz = 1.0 - qab * x / qap
    em = 0.0
    tem = 0.0
    d = 0.0
    ap = 0.0
    bp = 0.0
    app = 0.0
    bpp = 0.0
    aold = 0.0
    for m in range(1, itmax + 1):
        em = float(m)
        tem = em + em
        d = em * (b - em) * x / ((qam + tem) * (a + tem))
        ap = az + d * am
        bp = bz + d * bm
        d = -(a + em) * (qab + em) * x / ((a + tem) * (qap + tem))
        app = ap + d * az
        bpp = bp + d * bz
        aold = az
        am = ap / bpp
        bm = bp / bpp
        az = app / bpp
        bz = 1.0
        if abs(az - aold) < eps * abs(az):
            return az
    return az

def _betainc_reg(a: float, b: float, x: float) -> float:
    if x <= 0.0:
        return 0.0
    if x >= 1.0:
        return 1.0
    bt = math.exp(math.lgamma(a + b) - math.lgamma(a) - math.lgamma(b)
                  + a * math.log(x) + b * math.log(1.0 - x))
    if x < (a + 1.0) / (a + b + 2.0):
        return bt * _betacf(a, b, x) / a
    else:
        return 1.0 - bt * _betacf(b, a, 1.0 - x) / b

def f_cdf(x: float, d1: int, d2: int) -> float:
    """CDF of F(d1,d2) via I_{d1 x / (d1 x + d2)}(d1/2, d2/2)."""
    if x <= 0.0:
        return 0.0
    z = (d1 * x) / (d1 * x + d2)
    return _betainc_reg(0.5 * d1, 0.5 * d2, z)

# Levene's test
def _mean(a: Sequence[float]) -> float:
    return sum(a) / len(a)

def levene_test(groups: List[Sequence[float]]) -> Dict[str, Union[int, float]]:
    """
    Levene's test (center = mean). Tests equality of scale across k groups.
    Returns: F, df1, df2, p_value, N, k
    """
    # Flatten counts
    k = len(groups)
    ns = [len(g) for g in groups]
    if any(n < 2 for n in ns):
        raise ValueError("Each group must have at least 2 observations.")
    N = sum(ns)

    # Absolute deviations from group means
    z_groups = []
    for g in groups:
        mu = _mean(g)
        z_groups.append([abs(x - mu) for x in g])

    # One-way ANOVA on z
    z_means = [_mean(zg) for zg in z_groups]
    z_all = [z for zg in z_groups for z in zg]
    z_grand = _mean(z_all)

    # Between- and within-group sums of squares
    ss_between = sum(n * (m - z_grand) ** 2 for n, m in zip(ns, z_means))
    ss_within = sum(sum((z - m) ** 2 for z in zg) for zg, m in zip(z_groups, z_means))

    df1 = k - 1
    df2 = N - k
    ms_between = ss_between / df1
    ms_within = ss_within / df2
    F = ms_between / ms_within

    # Upper-tail p-value
    p = 1.0 - f_cdf(F, df1, df2)
    return {"F": F, "df1": df1, "df2": df2, "p_value": p, "N": N, "k": k}

if __name__ == "__main__":
    import numpy as np
    rng = np.random.default_rng(42)

    # Example: split a return series into 5 consecutive subperiods (groups)
    n = 250
    r = 0.0005 + 0.01 * rng.standard_t(df=3, size=n)  # heavy-tailed strategy returns
    G = 5
    groups = [r[i*(n//G):(i+1)*(n//G)] for i in range(G)]

    out = levene_test(groups)
    print(f"Levene: F={out['F']:.4f}, df=({out['df1']},{out['df2']}), p={out['p_value']:.6f}")

Levene: F=1.5778, df=(4,245), p=0.180795


Brown-Forsythe test

In [8]:
import math
from typing import List, Sequence, Dict, Union

# Regularized incomplete beta and F CDF
def _betacf(a: float, b: float, x: float, itmax: int = 200, eps: float = 1e-12) -> float:
    am, bm = 1.0, 1.0
    az = 1.0
    qab = a + b
    qap = a + 1.0
    qam = a - 1.0
    bz = 1.0 - qab * x / qap
    for m in range(1, itmax + 1):
        em = float(m)
        d = em * (b - em) * x / ((qam + 2*em) * (a + 2*em))
        ap = az + d * am
        bp = bz + d * bm
        d = -(a + em) * (qab + em) * x / ((a + 2*em) * (qap + 2*em))
        app = ap + d * az
        bpp = bp + d * bz
        aold = az
        am = ap / bpp
        bm = bp / bpp
        az = app / bpp
        bz = 1.0
        if abs(az - aold) < eps * abs(az):
            return az
    return az

def _betainc_reg(a: float, b: float, x: float) -> float:
    if x <= 0.0:
        return 0.0
    if x >= 1.0:
        return 1.0
    bt = math.exp(math.lgamma(a + b) - math.lgamma(a) - math.lgamma(b)
                  + a * math.log(x) + b * math.log(1.0 - x))
    if x < (a + 1.0) / (a + b + 2.0):
        return bt * _betacf(a, b, x) / a
    else:
        return 1.0 - bt * _betacf(b, a, 1.0 - x) / b

def f_cdf(x: float, d1: int, d2: int) -> float:
    if x <= 0.0:
        return 0.0
    z = (d1 * x) / (d1 * x + d2)
    return _betainc_reg(0.5 * d1, 0.5 * d2, z)

# Brown–Forsythe
def _median(a: Sequence[float]) -> float:
    s = sorted(a)
    n = len(s)
    mid = n // 2
    if n % 2 == 1:
        return s[mid]
    return 0.5 * (s[mid - 1] + s[mid])

def brown_forsythe_test(groups: List[Sequence[float]]) -> Dict[str, Union[int, float]]:
    """
    Brown–Forsythe test (center = median). Tests equality of scale across k groups.
    Returns: F, df1, df2, p_value, N, k
    """
    k = len(groups)
    ns = [len(g) for g in groups]
    if any(n < 2 for n in ns):
        raise ValueError("Each group must have at least 2 observations.")
    N = sum(ns)

    # Absolute deviations from group medians
    z_groups = []
    for g in groups:
        med = _median(g)
        z_groups.append([abs(x - med) for x in g])

    # One-way ANOVA on z
    def _mean(a): return sum(a) / len(a)
    z_means = [_mean(zg) for zg in z_groups]
    z_all = [z for zg in z_groups for z in zg]
    z_grand = _mean(z_all)

    ss_between = sum(n * (m - z_grand) ** 2 for n, m in zip(ns, z_means))
    ss_within = sum(sum((z - m) ** 2 for z in zg) for zg, m in zip(z_groups, z_means))

    df1 = k - 1
    df2 = N - k
    F = (ss_between / df1) / (ss_within / df2)

    p = 1.0 - f_cdf(F, df1, df2)
    return {"F": F, "df1": df1, "df2": df2, "p_value": p, "N": N, "k": k}

if __name__ == "__main__":
    import numpy as np
    rng = np.random.default_rng(7)

    # Same grouping idea as in Levene: 4 consecutive subperiods
    n = 240
    r = 0.0003 + 0.012 * rng.standard_t(df=3, size=n)

    G = 4
    groups = [r[i*(n//G):(i+1)*(n//G)] for i in range(G)]

    out = brown_forsythe_test(groups)
    print(f"Brown-Forsythe: F={out['F']:.4f}, df=({out['df1']},{out['df2']}), p={out['p_value']:.6f}")

Brown-Forsythe: F=0.8120, df=(3,236), p=0.488324


Kolmogorov–Smirnov (K–S) test

In [10]:
import math
from typing import Callable, Dict, Iterable, Tuple, Union
import numpy as np

# ---------- Utilities ----------
SQRT2 = math.sqrt(2.0)

def _erf_array(z: Union[float, np.ndarray]) -> np.ndarray:
    """Vectorized math.erf for arrays/scalars without SciPy."""
    z = np.asarray(z, dtype=float)
    # np.vectorize is fine here (sizes are small/moderate); replace with a fast approx if needed.
    return np.vectorize(math.erf)(z)

def normal_cdf(x: Union[float, np.ndarray], mu: float = 0.0, sigma: float = 1.0):
    z = (np.asarray(x, dtype=float) - mu) / sigma
    return 0.5 * (1.0 + _erf_array(z / SQRT2))

def _ks_asymp_pvalue(D: float, n_eff: float, tol: float = 1e-12) -> float:
    """
    Asymptotic K–S tail probability: P(D_n >= D) ≈ 2 * sum_{k>=1} (-1)^{k-1} exp(-2 k^2 n_eff D^2).
    For 1-sample, n_eff = n. For 2-sample, n_eff = n*m/(n+m).
    """
    if D <= 0.0:
        return 1.0
    x = -2.0 * (D * D) * n_eff
    s = 0.0
    k = 1
    while True:
        term = 2.0 * ((-1.0) ** (k - 1)) * math.exp(x * (k * k))
        s += term
        if abs(term) < tol:
            break
        k += 1
        if k > 5000:  # ultra-conservative guard
            break
    return max(0.0, min(1.0, s))

# ---------- One-sample K–S (known F) ----------
def ks_1samp(x: Iterable[float], cdf: Callable[[np.ndarray], np.ndarray]) -> Dict[str, float]:
    """
    One-sample K–S test against a fully specified continuous CDF F (no parameters fitted on x).
    Returns: D, D_plus, D_minus, n, p_value (asymptotic)
    """
    x = np.asarray(list(x), dtype=float)
    x = x[~np.isnan(x)]
    n = x.size
    if n == 0:
        raise ValueError("Empty sample.")

    xs = np.sort(x)
    Fi = cdf(xs)  # theoretical CDF at data points
    i = np.arange(1, n + 1)
    D_plus = np.max(i / n - Fi)
    D_minus = np.max(Fi - (i - 1) / n)
    D = max(D_plus, D_minus)

    p = _ks_asymp_pvalue(D, n_eff=n)
    return {"D": float(D), "D_plus": float(D_plus), "D_minus": float(D_minus),
            "n": int(n), "p_value": float(p)}

# ---------- Two-sample K–S ----------
def ks_2samp(x: Iterable[float], y: Iterable[float]) -> Dict[str, float]:
    """
    Two-sample K–S test comparing ECDFs of x and y.
    Returns: D, n, m, p_value (asymptotic with n_eff = n*m/(n+m))
    """
    x = np.asarray(list(x), dtype=float)
    y = np.asarray(list(y), dtype=float)
    x = x[~np.isnan(x)]
    y = y[~np.isnan(y)]
    n, m = x.size, y.size
    if n == 0 or m == 0:
        raise ValueError("Both samples must be non-empty.")

    xs = np.sort(x)
    ys = np.sort(y)
    z = np.sort(np.unique(np.concatenate([xs, ys])))   # merged grid
    Fx = np.searchsorted(xs, z, side="right") / n
    Fy = np.searchsorted(ys, z, side="right") / m
    D = float(np.max(np.abs(Fx - Fy)))
    n_eff = (n * m) / (n + m)

    p = _ks_asymp_pvalue(D, n_eff=n_eff)
    return {"D": D, "n": int(n), "m": int(m), "p_value": float(p)}

# ---------- Lilliefors (Monte Carlo) ----------
def lilliefors_normal(
    x: Iterable[float],
    nsim: int = 2000,
    rng: np.random.Generator = None,
    ddof_sigma_mle: int = 0,
) -> Dict[str, float]:
    """
    Lilliefors test for normality: K–S vs N(mu_hat, sigma_hat), with mu_hat, sigma_hat estimated from x.
    p-value obtained by Monte Carlo under H0.
    """
    if rng is None:
        rng = np.random.default_rng(12345)

    x = np.asarray(list(x), dtype=float)
    x = x[~np.isnan(x)]
    n = x.size
    if n < 3:
        raise ValueError("Sample too small for Lilliefors.")

    # Fit mu, sigma on the sample
    mu_hat = float(np.mean(x))
    sigma_hat = float(np.std(x, ddof=ddof_sigma_mle))
    if sigma_hat <= 0.0:
        return {"D": 1.0, "n": int(n), "p_value": 0.0, "nsim": int(nsim)}

    # Observed D against fitted normal
    xs = np.sort(x)
    Fi = 0.5 * (1.0 + _erf_array((xs - mu_hat) / (sigma_hat * SQRT2)))
    i = np.arange(1, n + 1)
    D_plus = np.max(i / n - Fi)
    D_minus = np.max(Fi - (i - 1) / n)
    D_obs = float(max(D_plus, D_minus))

    # Simulate null with parameter re-fitting each time
    geq = 0
    for _ in range(nsim):
        z = rng.standard_normal(n)
        mu_s = float(np.mean(z))
        sig_s = float(np.std(z, ddof=ddof_sigma_mle))
        if sig_s <= 0.0:
            continue
        zs = np.sort(z)
        Fsim = 0.5 * (1.0 + _erf_array((zs - mu_s) / (sig_s * SQRT2)))
        i = np.arange(1, n + 1)
        Dp = np.max(i / n - Fsim)
        Dm = np.max(Fsim - (i - 1) / n)
        D_sim = max(Dp, Dm)
        if D_sim >= D_obs:
            geq += 1

    p_mc = (geq + 1.0) / (nsim + 1.0)  # smoothed MC p-value
    return {"D": D_obs, "n": int(n), "p_value": float(p_mc), "nsim": int(nsim)}

if __name__ == "__main__":
    rng = np.random.default_rng(42)

    # Synthetic returns (fat-tailed) to show normal misspecification
    n = 300
    returns = 0.0003 + 0.01 * rng.standard_t(df=3, size=n)

    # 1) One-sample K–S against *fully specified* Normal(0, 1% daily vol)
    mu0, sig0 = 0.0, 0.01
    def F0(x):  # theoretical CDF known a priori (no parameter fitting!)
        return 0.5 * (1.0 + _erf_array((np.asarray(x) - mu0) / (sig0 * SQRT2)))

    ks1 = ks_1samp(returns, cdf=F0)
    print(f"[KS 1-sample vs N({mu0:.4f},{sig0:.4f}):] D={ks1['D']:.4f}, p={ks1['p_value']:.6f}, n={ks1['n']}")

    # 2) Lilliefors
    lillie = lilliefors_normal(returns, nsim=2000, rng=rng, ddof_sigma_mle=0)
    print(f"[Lilliefors normality:] D={lillie['D']:.4f}, p≈{lillie['p_value']:.6f} (nsim={lillie['nsim']})")

    # 3) Two-sample K–S: compare two subperiods (high-vol vs low-vol regime proxy)
    rA = returns[: n//2]
    rB = returns[n//2 :]
    ks2 = ks_2samp(rA, rB)
    print(f"[KS 2-sample A vs B:] D={ks2['D']:.4f}, p={ks2['p_value']:.6f}, n={ks2['n']}, m={ks2['m']}")


[KS 1-sample vs N(0.0000,0.0100):] D=0.0713, p=0.094391, n=300
[Lilliefors normality:] D=0.1215, p≈0.000500 (nsim=2000)
[KS 2-sample A vs B:] D=0.0533, p=0.983287, n=150, m=150


Chi Goodness-of-Fit test

In [11]:
import math
from typing import Callable, Dict, Iterable, List, Tuple, Union
import numpy as np

# Chi-square CDF via regularized incomplete gamma
def _gammainc_lower_reg(s: float, x: float, eps: float = 1e-12, itmax: int = 200) -> float:
    """Regularized lower incomplete gamma P(s,x) = gamma(s,x)/Gamma(s)."""
    if x <= 0.0:
        return 0.0
    if s <= 0.0:
        raise ValueError("s must be > 0")

    def _pref(ss, xx):
        return math.exp(ss * math.log(xx) - xx - math.lgamma(ss))

    if x < s + 1.0:
        # Series for P(s, x)
        term = 1.0 / s
        summ = term
        k = 1
        while k < itmax:
            term *= x / (s + k)
            summ += term
            if abs(term) < abs(summ) * eps:
                break
            k += 1
        return _pref(s, x) * summ
    else:
        # Continued fraction for Q(s, x) = 1 - P(s, x)
        tiny = 1e-300
        b = x - s + 1.0
        C = b if abs(b) > tiny else tiny
        D = 1.0 / (b if abs(b) > tiny else tiny)
        f = C * D
        for k in range(1, itmax + 1):
            a = -k * (k - s)
            b += 2.0
            D = a * D + b
            if abs(D) < tiny: D = tiny
            C = b + a / C
            if abs(C) < tiny: C = tiny
            D = 1.0 / D
            delta = C * D
            f *= delta
            if abs(delta - 1.0) < eps:
                break
        Q = math.exp(s * math.log(x) - x - math.lgamma(s)) * f
        P = 1.0 - Q
        return min(1.0, max(0.0, P))

def chi2_cdf(x: float, df: int) -> float:
    """CDF of Chi-square(df) using P(df/2, x/2)."""
    if df <= 0 or int(df) != df:
        raise ValueError("df must be a positive integer.")
    if x <= 0.0:
        return 0.0
    return _gammainc_lower_reg(0.5 * df, 0.5 * x)

# Binning rules & helpers
def _sturges_bins(x: np.ndarray) -> np.ndarray:
    n = x.size
    k = int(np.ceil(np.log2(n))) + 1 if n > 1 else 1
    lo, hi = np.min(x), np.max(x)
    if hi == lo:
        hi = lo + 1e-12
    return np.linspace(lo, hi, k + 1)

def _scott_bins(x: np.ndarray, max_bins: int = 500) -> np.ndarray:
    n = x.size
    sigma = np.std(x, ddof=1) if n > 1 else 0.0
    if sigma <= 0.0:
        return _sturges_bins(x)
    h = 3.49 * sigma * n ** (-1/3)
    lo, hi = np.min(x), np.max(x)
    if h <= 0 or hi == lo:
        return _sturges_bins(x)
    k = int(np.ceil((hi - lo) / h))
    k = max(1, min(k, max_bins))
    return np.linspace(lo, hi, k + 1)

def _fd_bins(x: np.ndarray, max_bins: int = 500) -> np.ndarray:
    n = x.size
    q75, q25 = np.percentile(x, [75, 25])
    iqr = q75 - q25
    if iqr <= 0:
        return _scott_bins(x, max_bins=max_bins)
    h = 2.0 * iqr * n ** (-1/3)
    lo, hi = np.min(x), np.max(x)
    if h <= 0 or hi == lo:
        return _sturges_bins(x)
    k = int(np.ceil((hi - lo) / h))
    k = max(1, min(k, max_bins))
    return np.linspace(lo, hi, k + 1)

def make_bin_edges(x: np.ndarray, rule: str = "fd", max_bins: int = 500) -> np.ndarray:
    rule = rule.lower()
    if rule in ("fd", "freedman-diaconis", "freedman_diaconis"):
        return _fd_bins(x, max_bins=max_bins)
    elif rule in ("scott",):
        return _scott_bins(x, max_bins=max_bins)
    elif rule in ("sturges",):
        return _sturges_bins(x)
    else:
        raise ValueError("Unknown rule. Use 'fd', 'scott', or 'sturges'.")

def _merge_low_expected(O: np.ndarray, E: np.ndarray, edges: np.ndarray, min_exp: float) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Greedily merge adjacent bins with E_i < min_exp. Prefer merging with the neighbor
    that yields the smallest combined expected count increase (left if i>0 else right).
    """
    O = O.astype(float).copy()
    E = E.astype(float).copy()
    edges = edges.astype(float).copy()

    i = 0
    while i < len(E):
        if E[i] >= min_exp:
            i += 1
            continue
        # choose neighbor
        if len(E) == 1:
            break
        if i == 0:
            j = 1
            left = False
        elif i == len(E) - 1:
            j = i - 1
            left = True
        else:
            # choose neighbor with smaller E
            left = E[i-1] <= E[i+1]
            j = i - 1 if left else i + 1
        # merge bin i into bin j (ensure j < i for edge deletion)
        if j > i:
            # merge into right neighbor
            O[j] += O[i]; E[j] += E[i]
            O = np.delete(O, i); E = np.delete(E, i)
            edges = np.delete(edges, i + 1)  # remove boundary between i and j
        else:
            # merge into left neighbor
            O[j] += O[i]; E[j] += E[i]
            O = np.delete(O, i); E = np.delete(E, i)
            edges = np.delete(edges, j + 1)  # remove boundary between j and i
            i = j  # current index moved left
    return O, E, edges

# Chi-squared GoF core
def chi2_gof_binned(
    x: Iterable[float],
    cdf: Callable[[np.ndarray], np.ndarray],
    bins: str = "fd",
    min_exp: float = 5.0,
    params_estimated: int = 0,
    max_bins: int = 500
) -> Dict[str, Union[int, float, np.ndarray]]:
    """
    Chi-squared GoF test against a fully specified CDF 'cdf'.
      - 'bins': 'fd' (Freedman–Diaconis), 'scott', or 'sturges'
      - 'min_exp': minimum expected count per bin (merges until satisfied)
      - 'params_estimated': number of parameters fitted on the data (df adjustment)
    Returns: chi2, df, p_value, k_final, O, E, edges
    """
    x = np.asarray(list(x), dtype=float)
    x = x[~np.isnan(x)]
    n = x.size
    if n < 2:
        raise ValueError("Not enough observations.")

    edges = make_bin_edges(x, rule=bins, max_bins=max_bins)
    # Observed counts
    O, _ = np.histogram(x, bins=edges)

    # Expected counts from theoretical CDF
    F_lo = cdf(edges[:-1])
    F_hi = cdf(edges[1:])
    probs = np.clip(F_hi - F_lo, 0.0, 1.0)
    E = n * probs

    # Merge bins until all E_i >= min_exp
    O2, E2, edges2 = _merge_low_expected(O, E, edges, min_exp=min_exp)

    # Recompute statistic
    if np.any(E2 <= 0):
        raise ValueError("Some expected counts are non-positive after merging.")
    chi2 = float(np.sum((O2 - E2) ** 2 / E2))

    k_final = int(len(E2))
    df = k_final - 1 - int(params_estimated)
    if df <= 0:
        raise ValueError(f"Non-positive degrees of freedom (df={df}). "
                         f"Reduce parameters_estimated or adjust binning.")

    p_value = 1.0 - chi2_cdf(chi2, df)
    return {
        "chi2": chi2,
        "df": df,
        "p_value": float(p_value),
        "k_final": k_final,
        "O": O2,
        "E": E2,
        "edges": edges2
    }

# Convenience: Normal CDF
SQRT2 = math.sqrt(2.0)
def _erf_array(z: Union[float, np.ndarray]) -> np.ndarray:
    z = np.asarray(z, dtype=float)
    return np.vectorize(math.erf)(z)

def normal_cdf(x: Union[float, np.ndarray], mu: float = 0.0, sigma: float = 1.0):
    z = (np.asarray(x, dtype=float) - mu) / sigma
    return 0.5 * (1.0 + _erf_array(z / SQRT2))

if __name__ == "__main__":
    rng = np.random.default_rng(7)

    # Synthetic returns (fat-tailed) to illustrate misspecification
    n = 300
    returns = 0.0003 + 0.01 * rng.standard_t(df=3, size=n)

    # Case A: Test against pre-specified Normal(0, 1% daily vol): p=0 parameters estimated
    mu0, sig0 = 0.0, 0.01
    F0 = lambda x: normal_cdf(x, mu=mu0, sigma=sig0)
    res_A = chi2_gof_binned(returns, cdf=F0, bins="fd", min_exp=5, params_estimated=0)
    print("[Chi2 GoF vs N(0, 0.01)]",
          f"chi2={res_A['chi2']:.3f}, df={res_A['df']}, p={res_A['p_value']:.6f}, k'={res_A['k_final']}")

    # Case B: Test against Normal with mu,sigma estimated from the sample: p=2
    mu_hat = float(np.mean(returns))
    sig_hat = float(np.std(returns, ddof=1))
    Fhat = lambda x: normal_cdf(x, mu=mu_hat, sigma=sig_hat)
    res_B = chi2_gof_binned(returns, cdf=Fhat, bins="fd", min_exp=5, params_estimated=2)
    print("[Chi2 GoF vs N(mu_hat, sigma_hat)]",
          f"chi2={res_B['chi2']:.3f}, df={res_B['df']}, p={res_B['p_value']:.6f}, k'={res_B['k_final']}")

[Chi2 GoF vs N(0, 0.01)] chi2=27.935, df=10, p=0.019028, k'=11
[Chi2 GoF vs N(mu_hat, sigma_hat)] chi2=34.673, df=12, p=0.006691, k'=15


Jarque-Bera (J-B) test

In [12]:
from typing import Dict, Iterable, Union
import numpy as np

# Chi-square CDF via regularized incomplete gamma
def _gammainc_lower_reg(s: float, x: float, eps: float = 1e-12, itmax: int = 200) -> float:
    """Regularized lower incomplete gamma P(s, x) = gamma(s, x) / Gamma(s)."""
    if x <= 0.0:
        return 0.0
    if s <= 0.0:
        raise ValueError("s must be > 0")

    def _pref(ss, xx):
        return math.exp(ss * math.log(xx) - xx - math.lgamma(ss))

    if x < s + 1.0:
        # Series for P(s, x)
        term = 1.0 / s
        summ = term
        for k in range(1, itmax + 1):
            term *= x / (s + k)
            summ += term
            if abs(term) < abs(summ) * eps:
                break
        return _pref(s, x) * summ
    else:
        # Continued fraction for Q(s, x) = 1 - P(s, x) (Lentz)
        tiny = 1e-300
        b = x - s + 1.0
        C = b if abs(b) > tiny else tiny
        D = 1.0 / (b if abs(b) > tiny else tiny)
        f = C * D
        for k in range(1, itmax + 1):
            a = -k * (k - s)
            b += 2.0
            D = a * D + b
            if abs(D) < tiny: D = tiny
            C = b + a / C
            if abs(C) < tiny: C = tiny
            D = 1.0 / D
            delta = C * D
            f *= delta
            if abs(delta - 1.0) < eps:
                break
        Q = math.exp(s * math.log(x) - x - math.lgamma(s)) * f
        P = 1.0 - Q
        return min(1.0, max(0.0, P))

def chi2_cdf(x: float, df: int) -> float:
    """CDF of Chi-square(df) using P(df/2, x/2)."""
    if df <= 0 or int(df) != df:
        raise ValueError("df must be a positive integer.")
    if x <= 0.0:
        return 0.0
    return _gammainc_lower_reg(0.5 * df, 0.5 * x)

# Jarque–Bera core
def _central_moments(x: np.ndarray) -> dict:
    """Return mean and central moments m2,m3,m4 using division by n (MLE-style)."""
    mu = float(np.mean(x))
    d = x - mu
    n = x.size
    m2 = float(np.mean(d**2))
    m3 = float(np.mean(d**3))
    m4 = float(np.mean(d**4))
    return {"mu": mu, "m2": m2, "m3": m3, "m4": m4}

def jarque_bera(
    x: Iterable[float],
    pvalue: str = "chi2",   # 'chi2' (asymptotic) or 'mc' (Monte Carlo)
    nsim: int = 5000,
    rng: np.random.Generator = None
) -> Dict[str, Union[int, float, str]]:
    """
    Jarque–Bera normality test (from scratch, no SciPy).
    - Moments use division by n (classical JB definition).
    - p-value via chi-square with 2 df ('chi2'), or Monte Carlo under Normal(0,1) ('mc').
    """
    x = np.asarray(list(x), dtype=float)
    x = x[~np.isnan(x)]
    n = x.size
    if n < 3:
        raise ValueError("Need at least 3 observations for JB.")

    cm = _central_moments(x)
    if cm["m2"] <= 0.0:
        # All values equal -> undefined skew/kurt; treat as extreme non-normal
        return {"n": n, "S": float("nan"), "K": float("nan"),
                "JB": float("inf"), "p_value": 0.0, "method": pvalue}

    S = cm["m3"] / (cm["m2"] ** 1.5)                # sample skewness (division by n)
    K = cm["m4"] / (cm["m2"] ** 2) - 3.0            # sample excess kurtosis (division by n)
    JB = n / 6.0 * (S**2 + 0.25 * K**2)

    if pvalue == "chi2":
        p = 1.0 - chi2_cdf(JB, df=2)
        method = "asymptotic-chi2(df=2)"
    elif pvalue == "mc":
        if rng is None:
            rng = np.random.default_rng(12345)
        # Monte Carlo null: standard normal samples of size n
        geq = 0
        for _ in range(nsim):
            z = rng.standard_normal(n)
            cmz = _central_moments(z)
            Sz = cmz["m3"] / (cmz["m2"] ** 1.5) if cmz["m2"] > 0 else 0.0
            Kz = cmz["m4"] / (cmz["m2"] ** 2) - 3.0 if cmz["m2"] > 0 else 0.0
            JBz = n / 6.0 * (Sz**2 + 0.25 * Kz**2)
            if JBz >= JB:
                geq += 1
        p = (geq + 1.0) / (nsim + 1.0)  # smoothed MC p-value
        method = f"monte-carlo (nsim={nsim})"
    else:
        raise ValueError("pvalue must be 'chi2' or 'mc'.")

    return {"n": n, "S": S, "K": K, "JB": JB, "p_value": float(p), "method": method}

if __name__ == "__main__":
    rng = np.random.default_rng(42)

    # (A) Nearly normal returns (tiny drift + ~1% vol)
    n = 300
    r_norm = 0.0003 + 0.01 * rng.standard_normal(n)
    outA = jarque_bera(r_norm, pvalue="chi2")
    print(f"[JB normal-ish] n={outA['n']}, S={outA['S']:.4f}, K={outA['K']:.4f}, "
          f"JB={outA['JB']:.3f}, p={outA['p_value']:.6f} ({outA['method']})")

    # (B) Heavy-tailed returns (Student-t df=3) to show rejection
    r_t = 0.0003 + 0.01 * rng.standard_t(df=3, size=n)
    outB = jarque_bera(r_t, pvalue="chi2")
    print(f"[JB t(3)]      n={outB['n']}, S={outB['S']:.4f}, K={outB['K']:.4f}, "
          f"JB={outB['JB']:.3f}, p={outB['p_value']:.6f} ({outB['method']})")

    # (C) Same sample as (B) but small-sample Monte Carlo p-value (optional)
    outB_mc = jarque_bera(r_t, pvalue="mc", nsim=3000, rng=rng)
    print(f"[JB t(3) MC]   n={outB_mc['n']}, JB={outB_mc['JB']:.3f}, "
          f"p≈{outB_mc['p_value']:.6f} ({outB_mc['method']})")

[JB normal-ish] n=300, S=0.2918, K=0.2264, JB=4.899, p=0.211511 (asymptotic-chi2(df=2))
[JB t(3)]      n=300, S=0.4962, K=12.0276, JB=1820.614, p=0.000000 (asymptotic-chi2(df=2))
[JB t(3) MC]   n=300, JB=1820.614, p≈0.000333 (monte-carlo (nsim=3000))


Two-sample mean test with autocorrelation (HAC-adjusted)

In [13]:
import math
from typing import Dict, Iterable, Tuple, Union
import numpy as np

# Utilities
SQRT2 = math.sqrt(2.0)
def _erf_array(z: Union[float, np.ndarray]) -> np.ndarray:
    z = np.asarray(z, dtype=float)
    return np.vectorize(math.erf)(z)
def normal_cdf(z: float) -> float:
    return 0.5 * (1.0 + _erf_array(z / SQRT2))

# HAC (Newey–West with Bartlett kernel)
def _autocovariances(x: np.ndarray, max_lag: int) -> Tuple[float, np.ndarray]:
    """
    Return (gamma_0, gamma[1:max_lag+1]) with division by n (not n-1).
    """
    x = np.asarray(x, dtype=float)
    x = x[~np.isnan(x)]
    n = x.size
    mu = np.mean(x)
    d = x - mu
    g0 = float(np.mean(d * d))
    if max_lag <= 0 or n <= 1:
        return g0, np.zeros(0)
    gammas = np.empty(max_lag, dtype=float)
    for k in range(1, max_lag + 1):
        gammas[k-1] = np.mean(d[k:] * d[:-k])
    return g0, gammas

def _nw_bandwidth(n: int) -> int:
    # Common rule-of-thumb: floor(4 * (n/100)^(2/9)), at least 1
    L = int(np.floor(4.0 * (n / 100.0) ** (2.0 / 9.0)))
    return max(1, L)

def hac_lrv(x: Iterable[float], max_lag: int = None) -> Tuple[float, float, int]:
    """
    Newey–West LRV with Bartlett weights: LRV = gamma0 + 2 * sum_{k=1}^L w_k * gamma_k,
    w_k = 1 - k/(L+1). Returns (LRV, sigma2_hat, L).
    """
    x = np.asarray(list(x), dtype=float)
    x = x[~np.isnan(x)]
    n = x.size
    if n < 2:
        raise ValueError("Need at least 2 observations.")
    if max_lag is None:
        max_lag = _nw_bandwidth(n)
    g0, g = _autocovariances(x, max_lag)
    L = max_lag
    if L == 0:
        lrv = g0
    else:
        k = np.arange(1, L + 1, dtype=float)
        w = 1.0 - k / (L + 1.0)  # Bartlett
        lrv = g0 + 2.0 * float(np.sum(w * g))
    # Guard against tiny negative due to numerical noise
    lrv = max(lrv, 1e-12)
    return lrv, g0, L

def mean_se_hac(x: Iterable[float], max_lag: int = None) -> Tuple[float, float, float, int]:
    """
    Returns (mean, SE_mean, sigma2_hat, L) where SE_mean = sqrt(LRV / n).
    """
    x = np.asarray(list(x), dtype=float)
    x = x[~np.isnan(x)]
    n = x.size
    mu = float(np.mean(x))
    lrv, sig2, L = hac_lrv(x, max_lag=max_lag)
    se = math.sqrt(lrv / n)
    return mu, se, sig2, L

def effective_sample_size(sig2: float, lrv: float, n: int) -> float:
    """
    n_eff = n * sig2 / lrv  (so that sig2 / n_eff = lrv / n).
    """
    return n * sig2 / max(lrv, 1e-12)

# Two-sample mean test with HAC
def two_sample_mean_test_hac(
    A: Iterable[float],
    B: Iterable[float],
    delta0: float = 0.0,           # hypothesized difference (mu_A - mu_B)
    alternative: str = "two-sided",
    lag_A: int = None,
    lag_B: int = None
) -> Dict[str, Union[float, int, str]]:
    """
    Test H0: mu_A - mu_B = delta0 using HAC (Newey–West) standard errors for means.
    Assumes independence across A and B. Returns z-stat (asymptotic) and p-value.
    """
    A = np.asarray(list(A), dtype=float); A = A[~np.isnan(A)]
    B = np.asarray(list(B), dtype=float); B = B[~np.isnan(B)]
    nA, nB = A.size, B.size
    if nA < 2 or nB < 2:
        raise ValueError("Both samples need at least 2 observations.")

    muA, seA, sig2A, LA = mean_se_hac(A, max_lag=lag_A)
    muB, seB, sig2B, LB = mean_se_hac(B, max_lag=lag_B)
    lrvA = (seA**2) * nA
    lrvB = (seB**2) * nB

    # Denominator via LRV (equivalently via n_eff)
    se_diff = math.sqrt(lrvA / nA + lrvB / nB)

    z = ((muA - muB) - delta0) / se_diff

    if alternative == "greater":         # H1: mu_A - mu_B > delta0
        p = 1.0 - float(normal_cdf(z))
    elif alternative == "less":          # H1: mu_A - mu_B < delta0
        p = float(normal_cdf(z))
    elif alternative == "two-sided":
        p = 2.0 * (1.0 - float(normal_cdf(abs(z))))
        p = min(1.0, max(0.0, p))
    else:
        raise ValueError("alternative must be 'two-sided','greater','less'.")

    n_eff_A = effective_sample_size(sig2A, lrvA, nA)
    n_eff_B = effective_sample_size(sig2B, lrvB, nB)

    return {
        "muA": muA, "muB": muB,
        "diff_hat": muA - muB,
        "z_stat": z, "p_value": p,
        "se_diff": se_diff,
        "nA": nA, "nB": nB,
        "L_A": LA, "L_B": LB,
        "sig2A": sig2A, "sig2B": sig2B,
        "lrvA": lrvA, "lrvB": lrvB,
        "n_eff_A": n_eff_A, "n_eff_B": n_eff_B,
        "alternative": alternative
    }

if __name__ == "__main__":
    rng = np.random.default_rng(42)
    n = 600

    # Construct two heavy-tailed, mildly autocorrelated return series
    # Strategy A: lower mean, phi=0.20
    eA = 0.01 * rng.standard_t(df=3, size=n)
    muA_true, phiA = 0.0004, 0.20
    A = np.empty(n); A[0] = muA_true + eA[0]
    for t in range(1, n):
        A[t] = muA_true + phiA * (A[t-1] - muA_true) + eA[t]

    # Strategy B: slightly higher mean, phi=0.10
    eB = 0.01 * rng.standard_t(df=3, size=n)
    muB_true, phiB = 0.0006, 0.10
    B = np.empty(n); B[0] = muB_true + eB[0]
    for t in range(1, n):
        B[t] = muB_true + phiB * (B[t-1] - muB_true) + eB[t]

    # Test H0: muA - muB = 0 (two-sided)
    out = two_sample_mean_test_hac(A, B, delta0=0.0, alternative="two-sided")
    print(f"muA={out['muA']:.6f}, muB={out['muB']:.6f}, diff={out['diff_hat']:.6f}")
    print(f"z={out['z_stat']:.3f}, p={out['p_value']:.6f}, SE(diff)={out['se_diff']:.6f}")
    print(f"n_eff_A≈{out['n_eff_A']:.1f}, n_eff_B≈{out['n_eff_B']:.1f} (L_A={out['L_A']}, L_B={out['L_B']})")

muA=0.000458, muB=0.000226, diff=0.000232
z=0.223, p=0.823782, SE(diff)=0.001042
n_eff_A≈510.6, n_eff_B≈519.6 (L_A=5, L_B=5)


Paired t-test with autocorrelation

In [14]:
from typing import Dict, Iterable, Tuple, Union
import numpy as np

# Utilities
SQRT2 = math.sqrt(2.0)
def _erf_array(z: Union[float, np.ndarray]) -> np.ndarray:
    z = np.asarray(z, dtype=float)
    return np.vectorize(math.erf)(z)

def normal_cdf(z: float) -> float:
    return 0.5 * (1.0 + _erf_array(z / SQRT2))

# HAC
def _autocovariances(x: np.ndarray, max_lag: int) -> Tuple[float, np.ndarray]:
    """
    gamma_0 and gamma_k (k=1..L), each divided by n (not n-1).
    """
    x = np.asarray(x, dtype=float)
    x = x[~np.isnan(x)]
    n = x.size
    mu = np.mean(x)
    d = x - mu
    g0 = float(np.mean(d * d))
    if max_lag <= 0 or n <= 1:
        return g0, np.zeros(0)
    gammas = np.empty(max_lag, dtype=float)
    for k in range(1, max_lag + 1):
        gammas[k-1] = np.mean(d[k:] * d[:-k])
    return g0, gammas

def _nw_bandwidth(n: int) -> int:
    # Rule-of-thumb bandwidth: floor(4 * (n/100)^(2/9)), at least 1
    L = int(np.floor(4.0 * (n / 100.0) ** (2.0 / 9.0)))
    return max(1, L)

def hac_lrv(x: Iterable[float], max_lag: int = None) -> Tuple[float, float, int]:
    """
    LRV = gamma0 + 2 * sum_{k=1}^L w_k * gamma_k, with Bartlett weights w_k = 1 - k/(L+1).
    Returns (LRV, sigma2_hat, L), where sigma2_hat = gamma0.
    """
    x = np.asarray(list(x), dtype=float)
    x = x[~np.isnan(x)]
    n = x.size
    if n < 2:
        raise ValueError("Need at least 2 observations.")
    if max_lag is None:
        max_lag = _nw_bandwidth(n)
    g0, g = _autocovariances(x, max_lag)
    if max_lag == 0:
        lrv = g0
    else:
        k = np.arange(1, max_lag + 1, dtype=float)
        w = 1.0 - k / (max_lag + 1.0)
        lrv = g0 + 2.0 * float(np.sum(w * g))
    return max(lrv, 1e-12), g0, max_lag  # guard tiny negatives

def mean_se_hac(x: Iterable[float], max_lag: int = None) -> Tuple[float, float, float, int]:
    """
    Mean and HAC standard error of the mean: SE = sqrt(LRV / n).
    Returns (mean, SE, sigma2_hat, L).
    """
    x = np.asarray(list(x), dtype=float)
    x = x[~np.isnan(x)]
    n = x.size
    mu = float(np.mean(x))
    lrv, sig2, L = hac_lrv(x, max_lag=max_lag)
    se = math.sqrt(lrv / n)
    return mu, se, sig2, L

def effective_sample_size(sig2: float, lrv: float, n: int) -> float:
    """
    n_eff = n * sig2 / lrv (so that sig2 / n_eff = lrv / n).
    """
    return n * sig2 / max(lrv, 1e-12)

# Paired HAC mean test
def paired_mean_test_hac(
    A: Iterable[float],
    B: Iterable[float],
    delta0: float = 0.0,          # hypothesized mean difference mu_B - mu_A
    alternative: str = "two-sided",
    lag: int = None
) -> Dict[str, Union[float, int, str]]:
    """
    Correct paired test using d_t = B_t - A_t (aligned in time) with HAC SE.
    Returns z-stat and p-value (asymptotic normal).
    """
    A = np.asarray(list(A), dtype=float)
    B = np.asarray(list(B), dtype=float)

    # Drop any pairs with NaN in either series (alignment in practice uses timestamps)
    mask = np.isfinite(A) & np.isfinite(B)
    A, B = A[mask], B[mask]
    if A.size != B.size or A.size < 2:
        raise ValueError("A and B must be aligned and have at least 2 paired observations.")

    d = B - A
    n = d.size
    mu_d, se_d, sig2_d, L = mean_se_hac(d, max_lag=lag)
    z = (mu_d - delta0) / se_d

    if alternative == "greater":      # H1: mu_B - mu_A > delta0
        p = 1.0 - float(normal_cdf(z))
    elif alternative == "less":       # H1: mu_B - mu_A < delta0
        p = float(normal_cdf(z))
    elif alternative == "two-sided":
        p = 2.0 * (1.0 - float(normal_cdf(abs(z))))
        p = min(1.0, max(0.0, p))
    else:
        raise ValueError("alternative must be 'two-sided','greater','less'.")

    n_eff = effective_sample_size(sig2_d, (se_d**2) * n, n)
    return {"mu_d": mu_d, "z_stat": z, "p_value": p, "se_mean": se_d,
            "n": n, "L": L, "sig2_d": sig2_d, "n_eff": n_eff,
            "alternative": alternative}

if __name__ == "__main__":
    rng = np.random.default_rng(2025)
    n = 500

    # Two strategies on the SAME dates, sharing a common market shock (positive covariance)
    muA, muB = 0.0004, 0.0006
    market = 0.008 * rng.standard_t(df=4, size=n)   # common factor, fat-tailed
    eA = 0.006 * rng.standard_normal(n)             # idiosyncratic noises
    eB = 0.006 * rng.standard_normal(n)

    A = muA + market + eA
    B = muB + market + eB

    # Correct paired HAC test (H1: B better than A)
    out = paired_mean_test_hac(A, B, delta0=0.0, alternative="greater")
    print(f"mu_d={out['mu_d']:.6f}, z={out['z_stat']:.3f}, p={out['p_value']:.6f}, "
          f"SE_mean={out['se_mean']:.6f}, n_eff≈{out['n_eff']:.1f} (L={out['L']})")

mu_d=0.000033, z=0.076, p=0.469536, SE_mean=0.000427, n_eff≈438.3 (L=5)


Testing positive serial correlation (trend-following)

In [17]:
    import math
    from typing import Callable, Dict, Iterable, Tuple, Union
    import numpy as np

    # Utilities
    SQRT2 = math.sqrt(2.0)
    def _erf_array(z: Union[float, np.ndarray]) -> np.ndarray:
        z = np.asarray(z, dtype=float)
        return np.vectorize(math.erf)(z)
    def normal_cdf(z: float) -> float:
        return float(0.5 * (1.0 + _erf_array(z / SQRT2)))

    # Chi-square CDF via regularized incomplete gamma (for Ljung–Box p)
    def _gammainc_lower_reg(s: float, x: float, eps: float = 1e-12, itmax: int = 200) -> float:
        """Regularized lower incomplete gamma P(s,x)."""
        if x <= 0.0:
            return 0.0
        if s <= 0.0:
            raise ValueError("s must be > 0")
        def _pref(ss, xx): return math.exp(ss*math.log(xx) - xx - math.lgamma(ss))
        if x < s + 1.0:
            term = 1.0 / s; summ = term
            for k in range(1, itmax + 1):
                term *= x / (s + k); summ += term
                if abs(term) < abs(summ) * eps: break
            return _pref(s, x) * summ
        else:
            tiny = 1e-300
            b = x - s + 1.0
            C = b if abs(b) > tiny else tiny
            D = 1.0 / (b if abs(b) > tiny else tiny)
            f = C * D
            for k in range(1, itmax + 1):
                a = -k * (k - s); b += 2.0
                D = a * D + b;  D = tiny if abs(D) < tiny else D
                C = b + a / C;  C = tiny if abs(C) < tiny else C
                D = 1.0 / D
                delta = C * D; f *= delta
                if abs(delta - 1.0) < eps: break
            Q = math.exp(s*math.log(x) - x - math.lgamma(s)) * f
            P = 1.0 - Q
            return min(1.0, max(0.0, P))

    def chi2_cdf(x: float, df: int) -> float:
        if df <= 0 or int(df) != df:
            raise ValueError("df must be a positive integer.")
        if x <= 0.0:
            return 0.0
        return _gammainc_lower_reg(0.5 * df, 0.5 * x)

    # Sample ACF and Ljung–Box Q
    def acf(x: Iterable[float], max_lag: int) -> np.ndarray:
        """
        Sample autocorrelation up to max_lag (rho_0..rho_max).
        Uses gamma_k = mean[(x_t-mu)(x_{t-k}-mu)] with division by n.
        """
        x = np.asarray(list(x), dtype=float)
        x = x[np.isfinite(x)]
        n = x.size
        if n < max_lag + 1:
            raise ValueError("Series too short for requested max_lag.")
        mu = np.mean(x)
        d = x - mu
        g0 = float(np.mean(d * d))
        rhos = np.empty(max_lag + 1, dtype=float)
        rhos[0] = 1.0
        for k in range(1, max_lag + 1):
            rhos[k] = float(np.mean(d[k:] * d[:-k])) / g0 if g0 > 0 else 0.0
        return rhos

    def ljung_box(x: Iterable[float], m: int = None) -> Dict[str, float]:
        """
        Ljung–Box Q = n(n+2) sum_{k=1}^m rho_k^2/(n-k) ~ Chi^2_m under H0 (white noise).
        """
        x = np.asarray(list(x), dtype=float)
        x = x[np.isfinite(x)]
        n = x.size
        if m is None:
            m = max(1, int(min(20, np.floor(np.sqrt(n)))))
        rhos = acf(x, m)
        Q = n * (n + 2.0) * float(np.sum((rhos[1:] ** 2) / (n - np.arange(1, m + 1))))
        p = 1.0 - chi2_cdf(Q, df=m)
        return {"Q": Q, "df": m, "p_value": float(p)}

    # AR(1) slope > 0 with HAC SE
    def _nw_bandwidth(n: int) -> int:
        # Common ROT bandwidth
        return max(1, int(np.floor(4.0 * (n / 100.0) ** (2.0 / 9.0))))

    def ar1_pos_trend_test_hac(x: Iterable[float], L: int = None) -> Dict[str, float]:
        """
        Test H0: phi <= 0 in r_t = alpha + phi r_{t-1} + u_t  vs H1: phi > 0.
        HAC covariance for (alpha, phi) via Newey–West (Bartlett). Returns z and p(one-sided).
        """
        x = np.asarray(list(x), dtype=float)
        x = x[np.isfinite(x)]
        y = x[1:]
        X = np.column_stack([np.ones_like(x[:-1]), x[:-1]])  # T x 2
        T = y.size
        if T < 5:
            raise ValueError("Series too short for AR(1) regression.")

        # OLS
        XtX = X.T @ X
        XtY = X.T @ y
        beta = np.linalg.solve(XtX, XtY)       # [alpha, phi]
        resid = y - X @ beta
        phi_hat = float(beta[1])

        # HAC sandwich: Var(beta) ≈ (1/T) Q^{-1} S Q^{-1}, Q = X'X/T
        if L is None:
            L = _nw_bandwidth(T)
        Z = X * resid[:, None]                  # T x 2, z_t = x_t u_t
        Q = XtX / T
        Q_inv = np.linalg.inv(Q)

        # S = Γ_0 + sum_{h=1}^L w_h (Γ_h + Γ_h')
        Gamma0 = (Z.T @ Z) / T
        S = Gamma0.copy()
        for h in range(1, L + 1):
            w = 1.0 - h / (L + 1.0)            # Bartlett weight
            Zh = Z[h:, :]
            Zlag = Z[:-h, :]
            Gamma_h = (Zh.T @ Zlag) / T
            S += w * (Gamma_h + Gamma_h.T)

        Var_beta = (Q_inv @ S @ Q_inv) / T
        se_phi = float(np.sqrt(max(Var_beta[1, 1], 1e-18)))

        z = (phi_hat - 0.0) / se_phi
        p_one_sided = 1.0 - normal_cdf(z)      # H1: phi > 0

        return {"phi_hat": phi_hat, "se_phi": se_phi, "z": z, "p_value": p_one_sided, "L": L, "T": T}

    if __name__ == "__main__":
        rng = np.random.default_rng(42)
        n = 600

        # Synthetic returns with trend-following (positive AR(1)) and heavy-tailed shocks
        phi = 0.15
        e = 0.01 * rng.standard_t(df=3, size=n)
        r = np.empty(n, dtype=float)
        r[0] = e[0]
        for t in range(1, n):
            r[t] = phi * r[t-1] + e[t]

        # (i) One-sided AR(1) slope > 0, HAC SE
        ar1 = ar1_pos_trend_test_hac(r)
        print(f"[AR(1) trend test] phi_hat={ar1['phi_hat']:.4f}, se={ar1['se_phi']:.4f}, "
              f"z={ar1['z']:.3f}, p(one-sided)={ar1['p_value']:.6f}, L={ar1['L']}")

        # (ii) Ljung–Box up to m lags
        lb = ljung_box(r, m=10)
        print(f"[Ljung–Box] Q={lb['Q']:.2f}, df={lb['df']}, p={lb['p_value']:.6f}")

[AR(1) trend test] phi_hat=0.1210, se=0.0387, z=3.125, p(one-sided)=0.000888, L=5
[Ljung–Box] Q=19.67, df=10, p=0.205435


Testing negative serial correlation (mean reversion)

In [18]:
    import math
    from typing import Dict, Iterable, Tuple, Union
    import numpy as np

    # Utilities
    SQRT2 = math.sqrt(2.0)
    def _erf_array(z: Union[float, np.ndarray]) -> np.ndarray:
        z = np.asarray(z, dtype=float)
        return np.vectorize(math.erf)(z)
    def normal_cdf(z: float) -> float:
        return float(0.5 * (1.0 + _erf_array(z / SQRT2)))

    # HAC
    def _nw_bandwidth(n: int) -> int:
        # Rule-of-thumb bandwidth: floor(4 * (n/100)^(2/9)), at least 1
        return max(1, int(np.floor(4.0 * (n / 100.0) ** (2.0 / 9.0))))

    def ar1_neg_meanrev_test_hac(x: Iterable[float], L: int = None) -> Dict[str, float]:
        """
        Test H0: phi >= 0 vs H1: phi < 0 in r_t = alpha + phi r_{t-1} + u_t.
        HAC covariance for (alpha, phi) via Newey–West (Bartlett). Returns z and p(one-sided).
        """
        x = np.asarray(list(x), dtype=float)
        x = x[np.isfinite(x)]
        y = x[1:]
        X = np.column_stack([np.ones_like(x[:-1]), x[:-1]])  # T x 2
        T = y.size
        if T < 5:
            raise ValueError("Series too short for AR(1) regression.")

        # OLS estimates
        XtX = X.T @ X
        XtY = X.T @ y
        beta = np.linalg.solve(XtX, XtY)       # [alpha, phi]
        resid = y - X @ beta
        phi_hat = float(beta[1])

        # HAC sandwich: Var(beta) ≈ (1/T) Q^{-1} S Q^{-1}
        if L is None:
            L = _nw_bandwidth(T)
        Z = X * resid[:, None]                  # T x 2, z_t = x_t u_t
        Q = XtX / T
        Q_inv = np.linalg.inv(Q)

        # S = Γ_0 + sum_{h=1}^L w_h (Γ_h + Γ_h')
        Gamma0 = (Z.T @ Z) / T
        S = Gamma0.copy()
        for h in range(1, L + 1):
            w = 1.0 - h / (L + 1.0)            # Bartlett weight
            Zh = Z[h:, :]
            Zlag = Z[:-h, :]
            Gamma_h = (Zh.T @ Zlag) / T
            S += w * (Gamma_h + Gamma_h.T)

        Var_beta = (Q_inv @ S @ Q_inv) / T
        se_phi = float(np.sqrt(max(Var_beta[1, 1], 1e-18)))

        z = (phi_hat - 0.0) / se_phi
        # One-sided p for mean reversion: H1: phi < 0  =>  P(Z <= z)
        p_one_sided = float(normal_cdf(z))

        return {"phi_hat": phi_hat, "se_phi": se_phi, "z": z, "p_value": p_one_sided, "L": L, "T": T}

    # Heteroskedasticity-robust variance ratio
    def variance_ratio_test(r: Iterable[float], q: int = 5) -> Dict[str, float]:
        """
        Lo–MacKinlay variance–ratio test with heteroskedasticity-robust variance.
        H0: random walk (VR=1). Mean reversion => VR < 1 (one-sided 'less').
        Returns VR, Z (two-sided), and p_one_sided_less for VR<1.
        """
        r = np.asarray(list(r), dtype=float)
        r = r[np.isfinite(r)]
        n = r.size
        if q < 2 or n <= q:
            raise ValueError("Need q>=2 and n>q.")

        mu = float(np.mean(r))
        d = r - mu
        # Variance of 1-step returns (division by n, consistent with asymptotics)
        s2_1 = float(np.mean(d**2))
        if s2_1 <= 0.0:
            return {"VR": 1.0, "Z": 0.0, "p_two_sided": 1.0, "p_one_sided_less": 0.5, "q": q}

        # Overlapping q-period returns via cumulative sums
        c = np.cumsum(np.insert(r, 0, 0.0))
        y = c[q:] - c[:-q]                      # length m = n - q + 1
        m = y.size
        # Center y around q*mu to remove mean
        y_center = y - q * mu
        s2_q = float(np.mean(y_center**2))

        VR = s2_q / (q * s2_1)

        # Heteroskedasticity-robust variance:
        # theta(q) = sum_{j=1}^{q-1} [ (2(q-j)/q)^2 * delta_j ]
        # delta_j = sum_{t=j}^{n-1} (d_t^2 * d_{t-j}^2) / (sum_{t=0}^{n-1} d_t^2)^2
        denom = float(np.sum(d**2)) ** 2
        theta = 0.0
        for j in range(1, q):
            num = float(np.sum(d[j:]**2 * d[:-j]**2))
            delta_j = num / denom
            wj = 2.0 * (q - j) / q
            theta += (wj ** 2) * delta_j

        # Asymptotic: sqrt(n) (VR - 1) -> N(0, theta)
        Z = (np.sqrt(n) * (VR - 1.0)) / math.sqrt(max(theta, 1e-18))
        p_two_sided = 2.0 * (1.0 - normal_cdf(abs(Z)))
        p_two_sided = min(1.0, max(0.0, p_two_sided))
        # One-sided (mean reversion): VR < 1  =>  Z < 0
        p_one_sided_less = float(normal_cdf(Z))

        return {"VR": VR, "Z": float(Z), "p_two_sided": float(p_two_sided),
                "p_one_sided_less": p_one_sided_less, "q": q}

    if __name__ == "__main__":
        rng = np.random.default_rng(7)
        n = 800

        # Synthetic mean-reverting returns: AR(1) with phi < 0 and fat-tailed shocks
        phi = -0.20
        e = 0.01 * rng.standard_t(df=3, size=n)
        r = np.empty(n, dtype=float)
        r[0] = e[0]
        for t in range(1, n):
            r[t] = phi * r[t-1] + e[t]

        # (i) AR(1) one-sided test for phi < 0 (mean reversion)
        ar1 = ar1_neg_meanrev_test_hac(r)
        print(f"[AR(1) mean-reversion] phi_hat={ar1['phi_hat']:.4f}, se={ar1['se_phi']:.4f}, "
              f"z={ar1['z']:.3f}, p(one-sided)={ar1['p_value']:.6f}, L={ar1['L']}")

        # (ii) Variance–Ratio test at horizon q (VR<1 indicates mean reversion)
        vr = variance_ratio_test(r, q=10)
        print(f"[Variance–Ratio q=10] VR={vr['VR']:.4f}, Z={vr['Z']:.3f}, "
              f"p_two_sided={vr['p_two_sided']:.6f}, p_one_sided(VR<1)={vr['p_one_sided_less']:.6f}")

[AR(1) mean-reversion] phi_hat=-0.1455, se=0.0338, z=-4.300, p(one-sided)=0.000009, L=6
[Variance–Ratio q=10] VR=0.6862, Z=-74.533, p_two_sided=0.000000, p_one_sided(VR<1)=0.000000


Ljung–Box portmanteau test

In [19]:
import math
from typing import Dict, Iterable, Optional
import numpy as np

# Minimal normal CDF
def _normal_cdf(z: float) -> float:
    return 0.5 * (1.0 + math.erf(z / math.sqrt(2.0)))

def _default_m(n: int) -> int:
    # Rule of thumb mentioned in text: m ≈ ln(n)
    return max(1, int(round(math.log(n))))

def ljung_box(
    x: Iterable[float],
    m: Optional[int] = None,
    center: bool = True,
    pval: str = "chi2",          # 'chi2' (Wilson–Hilferty) or 'mc'
    nsim: int = 2000,
    rng: Optional[np.random.Generator] = None
) -> Dict[str, float]:
    """
    Ljung–Box Q = n(n+2) * sum_{k=1}^m (rho_k^2 / (n-k)).
    p-values:
      - 'chi2' : Wilson–Hilferty approximation to Chi^2_m
      - 'mc'   : Monte Carlo under N(0,1) white noise (same n)
    """
    x = np.asarray(list(x), dtype=float)
    x = x[np.isfinite(x)]
    n = x.size
    if n < 5:
        raise ValueError("Series too short for Ljung–Box.")
    if m is None:
        m = _default_m(n)

    # Center if requested
    d = x - np.mean(x) if center else x.copy()
    denom = float(np.dot(d, d))
    if denom <= 0.0:
        return {"Q": 0.0, "df": m, "p_value": 1.0, "method": "degenerate"}

    # Autocorrelations rho_1..rho_m (using denominator sum d^2)
    rhos = np.empty(m, dtype=float)
    for k in range(1, m + 1):
        rhos[k-1] = float(np.dot(d[k:], d[:-k])) / denom if n > k else 0.0

    Q = n * (n + 2.0) * float(np.sum((rhos ** 2) / (n - np.arange(1, m + 1))))

    if pval == "chi2":
        # Wilson–Hilferty cube-root normal approximation for Chi^2_m tail
        df = m
        if Q <= 0.0:
            p = 1.0
        else:
            z = ((Q / df) ** (1.0 / 3.0) - (1.0 - 2.0 / (9.0 * df))) / math.sqrt(2.0 / (9.0 * df))
            p = 1.0 - _normal_cdf(z)  # upper tail
        method = "chi2 (Wilson–Hilferty)"
    elif pval == "mc":
        if rng is None:
            rng = np.random.default_rng(12345)
        ge = 0
        for _ in range(nsim):
            z = rng.standard_normal(n)
            dz = z - z.mean() if center else z
            denom_z = float(np.dot(dz, dz))
            if denom_z <= 0.0:
                continue
            rhos_z = []
            for k in range(1, m + 1):
                rhos_z.append(float(np.dot(dz[k:], dz[:-k])) / denom_z if n > k else 0.0)
            Qz = n * (n + 2.0) * float(np.sum((np.array(rhos_z) ** 2) / (n - np.arange(1, m + 1))))
            if Qz >= Q:
                ge += 1
        p = (ge + 1.0) / (nsim + 1.0)
        method = f"monte-carlo (nsim={nsim})"
    else:
        raise ValueError("pval must be 'chi2' or 'mc'.")

    return {"Q": float(Q), "df": int(m), "p_value": float(min(max(p, 0.0), 1.0)), "method": method}

if __name__ == "__main__":
    rng = np.random.default_rng(42)
    n = 600

    # (A) White noise (null): should NOT reject
    r_null = 0.01 * rng.standard_normal(n)
    outA = ljung_box(r_null, m=10, pval="chi2")
    print(f"[Null] Q={outA['Q']:.2f}, df={outA['df']}, p={outA['p_value']:.4f} ({outA['method']})")

    # (B) AR(1) persistence (trend-like): likely to reject
    phi = 0.20
    e = 0.01 * rng.standard_t(df=4, size=n)
    r = np.empty(n); r[0] = e[0]
    for t in range(1, n):
        r[t] = phi * r[t-1] + e[t]
    outB = ljung_box(r, m=10, pval="chi2")
    print(f"[AR(1) phi=0.20] Q={outB['Q']:.2f}, df={outB['df']}, p={outB['p_value']:.6f} ({outB['method']})")

    # (C) Volatility clustering with little mean autocorr: test on squared returns
    # Simple GARCH-like volatility; returns are mostly uncorrelated, but r^2 is
    omega, alpha, beta = 1e-6, 0.10, 0.86
    sig2 = np.empty(n); sig2[0] = 1e-4
    z = rng.standard_normal(n)
    r_vol = np.empty(n)
    for t in range(n):
        if t > 0:
            sig2[t] = omega + alpha * (r_vol[t-1] ** 2) + beta * sig2[t-1]
        r_vol[t] = math.sqrt(sig2[t]) * z[t]

    # Ljung–Box on returns (often insignificant) vs on squared, which shows dependence
    outC_r = ljung_box(r_vol, m=10, pval="chi2")
    outC_s = ljung_box((r_vol**2 - np.mean(r_vol**2)), m=10, pval="chi2")
    print(f"[Vol cluster] r_t:  Q={outC_r['Q']:.2f}, p={outC_r['p_value']:.4f} | "
          f"(r_t^2): Q={outC_s['Q']:.2f}, p={outC_s['p_value']:.6f}")

[Null] Q=13.59, df=10, p=0.1919 (chi2 (Wilson–Hilferty))
[AR(1) phi=0.20] Q=24.90, df=10, p=0.005659 (chi2 (Wilson–Hilferty))
[Vol cluster] r_t:  Q=10.41, p=0.4050 | (r_t^2): Q=197.50, p=0.000000


Engle–Granger two-step cointegration test

In [22]:
import numpy as np
from typing import Dict, Iterable, Optional

def _ols_beta(X: np.ndarray, y: np.ndarray):
    # OLS via least squares; returns beta, residuals, sigma2 (SSR/T), and (X'X)^(-1)
    beta, *_ = np.linalg.lstsq(X, y, rcond=None)
    resid = y - X @ beta
    T, k = X.shape
    SSR = float(resid.T @ resid)
    sigma2 = SSR / T
    XtX_inv = np.linalg.inv(X.T @ X)
    return beta, resid, sigma2, XtX_inv

def _adf_design(resid: np.ndarray, p: int):
    """
    Build ADF design for Δe_t = γ e_{t-1} + Σ_{i=1}^p φ_i Δe_{t-i} + u_t (no intercept).
    Returns Y (length T-p) and X with columns [e_{t-1}, Δe_{t-1}, ..., Δe_{t-p}].
    """
    e = resid
    de = np.diff(e)          # length T = n-1
    T = de.size
    if p < 0 or p > T - 1:
        raise ValueError("Invalid lag order p for ADF design.")

    # Target vector: Δe_t for t = p+1..T  => indices p..T-1
    Y = de[p:]               # length T - p

    # Regressor 1: e_{t-1} aligned with Y  => e indices p..T-1 (i.e., e[p:T])
    col_e = e[p:T][:, None]  # length T - p

    if p == 0:
        return Y, col_e

    # Regressors 2..(p+1): Δe_{t-i}, i=1..p, aligned with Y
    lag_cols = [de[p - i : T - i] for i in range(1, p + 1)]  # each length T - p
    X = np.column_stack([col_e] + lag_cols)
    return Y, X

def _bic(SSR: float, T_eff: int, k: int) -> float:
    # BIC = T * ln(SSR/T) + k * ln(T)
    return T_eff * np.log(SSR / T_eff) + k * np.log(T_eff)

def _select_adf_lag(resid: np.ndarray, p_max: int) -> int:
    # Choose p in 0..p_max by BIC
    best_p, best_bic = 0, np.inf
    for p in range(p_max + 1):
        Y, X = _adf_design(resid, p)
        T_eff, k = X.shape
        beta, r, _, _ = _ols_beta(X, Y)
        SSR = float(r.T @ r)
        val = _bic(SSR, T_eff, k)
        if val < best_bic:
            best_bic, best_p = val, p
    return best_p

def engle_granger(
    y: Iterable[float],
    x: Iterable[float],
    p_max: Optional[int] = None,
    nsim: int = 2000,
    rng: Optional[np.random.Generator] = None
) -> Dict[str, float]:
    """
    Engle–Granger two-step cointegration test with BIC lag selection for ADF
    and Monte Carlo p-value under the null (independent random walks).
    Returns slope (hedge ratio), adf_t, chosen lag p, and p_mc.
    """
    if rng is None:
        rng = np.random.default_rng(12345)
    y = np.asarray(list(y), dtype=float)
    x = np.asarray(list(x), dtype=float)
    mask = np.isfinite(y) & np.isfinite(x)
    y, x = y[mask], x[mask]
    n = y.size
    if n < 30:
        raise ValueError("Need at least ~30 observations for a meaningful EG test.")

    # Step 1: OLS Y on [1, X]
    X1 = np.column_stack([np.ones(n), x])
    b, e, _, _ = _ols_beta(X1, y)  # b[1] is hedge ratio
    beta0, beta1 = float(b[0]), float(b[1])

    # Step 2: ADF on residuals (no intercept)
    if p_max is None:
        p_max = max(0, min(12, int(np.floor(12 * (n / 100.0) ** 0.25))))  # compact ROT
    p = _select_adf_lag(e, p_max)
    Y, X = _adf_design(e, p)
    T_eff, k = X.shape
    beta_adf, resid_adf, SSR, XtX_inv = _ols_beta(X, Y)
    # t-stat for gamma = coefficient on e_{t-1}
    sigma_hat2 = float((resid_adf.T @ resid_adf) / (T_eff - k))
    se = float(np.sqrt(sigma_hat2 * XtX_inv[0, 0]))
    gamma = float(beta_adf[0])
    t_stat = gamma / se

    # Monte Carlo p-value under null: X, Y are independent RW, same n
    # For each sim: regress Y on X, build residuals, run ADF with the same p_max and lag selection.
    # Tail is "more negative than observed" => evidence for stationarity.
    count = 0
    for _ in range(nsim):
        # Random walks with unit Gaussian innovations
        u = rng.standard_normal(n)
        v = rng.standard_normal(n)
        Xrw = np.cumsum(u)
        Yrw = np.cumsum(v)
        B, E, _, _ = _ols_beta(np.column_stack([np.ones(n), Xrw]), Yrw)
        # ADF on residuals
        ps = _select_adf_lag(E, p_max)
        Ys, Xs = _adf_design(E, ps)
        Ts, ks = Xs.shape
        bet_s, r_s, _, XtX_inv_s = _ols_beta(Xs, Ys)
        sig2_s = float((r_s.T @ r_s) / (Ts - ks))
        se_s = float(np.sqrt(sig2_s * XtX_inv_s[0, 0]))
        t_s = float(bet_s[0] / se_s)
        if t_s <= t_stat:
            count += 1
    p_mc = (count + 1.0) / (nsim + 1.0)

    return {
        "beta0": beta0,
        "beta1": beta1,
        "adf_t": t_stat,
        "lag_p": int(p),
        "p_value_mc": float(p_mc),
        "n": int(n)
    }

if __name__ == "__main__":
    rng = np.random.default_rng(7)
    n = 600

    # (A) Cointegrated pair: Y = 0.5 + 2.0 * X + u_t,  u_t stationary AR(1)
    eps = rng.standard_normal(n)
    X = np.cumsum(rng.standard_normal(n))                        # random walk
    u = np.empty(n); u[0] = eps[0] * 0.3
    phi = 0.6
    for t in range(1, n):
        u[t] = phi * u[t-1] + 0.3 * eps[t]                      # stationary
    Y = 0.5 + 2.0 * X + u
    eg_ok = engle_granger(Y, X, nsim=1500, rng=rng)
    print(f"[Cointegrated] beta1≈{eg_ok['beta1']:.3f}, t_adf={eg_ok['adf_t']:.3f}, "
          f"p_mc≈{eg_ok['p_value_mc']:.4f}, p={eg_ok['lag_p']}")

    # (B) Spurious correlation: independent random walks
    Xs = np.cumsum(rng.standard_normal(n))
    Ys = np.cumsum(rng.standard_normal(n))
    eg_spur = engle_granger(Ys, Xs, nsim=1500, rng=rng)
    print(f"[Spurious RW]  beta1≈{eg_spur['beta1']:.3f}, t_adf={eg_spur['adf_t']:.3f}, "
          f"p_mc≈{eg_spur['p_value_mc']:.4f}, p={eg_spur['lag_p']}")

[Cointegrated] beta1≈2.002, t_adf=-12.536, p_mc≈0.0007, p=0
[Spurious RW]  beta1≈-0.754, t_adf=-1.773, p_mc≈0.6356, p=0
